In [1]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import sys
import papermill as pm
import getpass

In [2]:
# import data_set_analysis_helper_functions as ahf

In [3]:
sys.executable

'/anaconda3/bin/python3'

In [4]:
sys.path[0]

'/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot'

## Path to data set: (change based on analysis)

In [5]:
path = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/"

# folder for analysis type: (change based on analysis)

In [6]:
#folderName="right_dual_color_photo_full_data_after_QC" 
folderName="left_dual_color_photo_full_data_after_QC" 

## what data set we are working on: (change based on analysis)

In [7]:
seq_str = '0a1b2b3d4abcd5a6a7a'
HowManyBack=1
side_of_brain = "left_side_of_the_brain"
combinations_of_sessions = "avg_of_mice"

In [8]:
path_to_load_data = path+"/"+folderName+"/"+seq_str
path_to_load_data

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis//left_dual_color_photo_full_data_after_QC/0a1b2b3d4abcd5a6a7a'

# list of y axis 

In [9]:
y_axis_list = ["df_f","z_score","z_score_sliding_window","z_score_smooth"]

## Load master spread sheet

In [10]:
#MasterSheetName = "3.lateral_ medial_6_lateral_channels_final"
MasterSheetName = "4.lateral_ medial_6_medial_channels_final"
Mouse_Date_FileName = pd.read_excel(path+'/'+folderName+"/"+MasterSheetName+'.xlsx')

In [11]:
Mouse_Date_FileName

,mouse,date,file name
0,T163,181127,T163-144959
1,T163,181207,T163-123432
2,T163,181128,T163-115220
3,T163,181204,T163-114628
4,T163,181206,T163-112720
5,T163,181210,T163-102536
6,T163,181213,T163-094646
7,T164,181127,T164-153101
8,T164,181203,T164-122950
9,T164,181128,T164-122054


In [12]:
def session_range_plus1(Mouse_Date_FileName):
    session_length = []
    mice = Mouse_Date_FileName['mouse'].unique() 
    for m in mice:
        index = Mouse_Date_FileName[Mouse_Date_FileName['mouse']==m].index[-1]
        session_length.append(index)
    session_length = list(map(lambda x:x+1, session_length))
    session_length.insert(0, 0)
    return session_length

In [13]:
#makes a list of index numbers, each corresponding to the last session of the mouse from mice list
sessionIndex_perMouse = session_range_plus1 (Mouse_Date_FileName)

In [14]:
sessionIndex_perMouse = list(map (str,sessionIndex_perMouse)) #turn it into a string.. 
# Could not pass the the list with papermill so turned it into a string. In the notebook itself i turn it back 
#into a list of integers. 

In [15]:
sessionIndex_perMouse

['0', '7', '10', '22', '26', '44', '52', '56', '59']

# list of periods to plot: 

In [16]:
list_of_periods=['combined_periods_1']

In [17]:
# list_of_periods=['Real_nTrials',
#                  'Reward_NoReward_tag',
#                  'ENL_tag',
#                  'Cue_to_Sel_tag',
#                  'Cue_to_Sel_fromPrevTrial_tag',
#                  'firstENLp_tag',
#                  'pre_Reward_NoReward_tag',
#                  'post_Reward_NoReward_tag',
#                  'pre_ENL_tag',
#                  'post_ENL_tag',
#                  'pre_Cue_to_Sel_tag',
#                  'post_Cue_to_Sel_tag',
#                  'pre_Cue_to_Sel_fromPrevTrial_tag',
#                  'post_Cue_to_Sel_fromPrevTrial_tag',
#                  'pre_firstENLp_tag',
#                  'post_firstENLp_tag',
#                  'first_600_bins_Reward_NoReward_tag',
#                  'last_200_bins_ENL_tag',
#                  'first_15_bins_Cue_to_Sel_tag',
#                  'first_15_bins_Cue_to_Sel_fromPrevTrial_tag',
#                  'first_20_bins_firstENLp_tag',
#                  'combined_periods_1']

# list of notebooks: 

In [ ]:
# nb_to_run_list = ["0.dsl__winall(no_dirc)_loseall(no_dirc)__rs_mice.ipynb",       
#                   "1.dsl__ipsinext_vs_contranext__rs_mice.ipynb",
#                   "2.dsl__ipsiprevious_vs_contraprevious__rs_mice.ipynb",
#                   "3.dsl__winall(ipsi_vs_contra_next)_loseall(ipsi_vs_contra_next)__rs_mice.ipynb",
#                   "4.dsl__winall(ipsi_vs_contra_previous)_loseall(ipsi_vs_contra_previous)__rs_mice.ipynb",
#                   "5.dsl__winrepeat(ipsi_vs_contra_next)_loseswitch(ipsi_vs_contra_next)__rs_mice.ipynb",
#                   "6.dsl__winrepeat(ipsi_vs_contra_previous)_loseswitch(ipsi_vs_contra_previous)__rs_mice.ipynb",
#                   "7.dsl__ipsi_next(WR_vs_LS)_contra_next(WR_vs_LS)__rs_mice.ipynb",
#                   "8.dsl__ipsi_previous(WR_vs_LS)_contra_previous(WR_vs_LS)__rs_mice.ipynb"]

In [18]:
nb_to_run_list = ["0.dsl__winall(no_dirc)_loseall(no_dirc)__ls_mice.ipynb",
                  "1.dsl__ipsinext_vs_contranext__ls_mice.ipynb",
                  "2.dsl__ipsiprevious_vs_contraprevious__ls_mice.ipynb",
                  "3.dsl__winall(ipsi_vs_contra_next)_loseall(ipsi_vs_contra_next)__ls_mice.ipynb",
                  "4.dsl__winall(ipsi_vs_contra_previous)_loseall(ipsi_vs_contra_previous)__ls_mice.ipynb",
                  "5.dsl__winrepeat(ipsi_vs_contra_next)_loseswitch(ipsi_vs_contra_next)__ls_mice.ipynb",
                  "6.dsl__winrepeat(ipsi_vs_contra_previous)_loseswitch(ipsi_vs_contra_previous)__ls_mice.ipynb",
                  "7.dsl__ipsi_next(WR_vs_LS)_contra_next(WR_vs_LS)__ls_mice.ipynb",
                  "8.dsl__ipsi_previous(WR_vs_LS)_contra_previous(WR_vs_LS)__ls_mice.ipynb"]

In [19]:
#parameters that were defined above and will carry on for the whole analysis. 
dict_for_pm = dict(seq_str= seq_str, 
                   HowManyBack = HowManyBack,
                   folderName = folderName,
                   path_to_load_data=path_to_load_data,
                   sessionIndex_perMouse = sessionIndex_perMouse,
                   y_axis_list = y_axis_list)     

In [20]:
# dict_for_pm["sessionIndex_perMouse"]

In [21]:
%%time
for nb_to_run in nb_to_run_list: 
    
    nb_to_run_with_pm = sys.path[0]+"/"+side_of_brain+"/"+combinations_of_sessions+"/"+nb_to_run
    print (nb_to_run_with_pm)
    for period_of_interest in list_of_periods:
        print (period_of_interest)
        dict_for_pm["period_of_interest"]=period_of_interest 
        pm.execute_notebook(nb_to_run_with_pm,
                            'TestOutPut.ipynb',kernel_name = "python3",
                            parameters= dict_for_pm,nest_asyncio=True)

/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/0.dsl__winall(no_dirc)_loseall(no_dirc)__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/1.dsl__ipsinext_vs_contranext__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/2.dsl__ipsiprevious_vs_contraprevious__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/3.dsl__winall(ipsi_vs_contra_next)_loseall(ipsi_vs_contra_next)__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/4.dsl__winall(ipsi_vs_contra_previous)_loseall(ipsi_vs_contra_previous)__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/5.dsl__winrepeat(ipsi_vs_contra_next)_loseswitch(ipsi_vs_contra_next)__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/6.dsl__winrepeat(ipsi_vs_contra_previous)_loseswitch(ipsi_vs_contra_previous)__ls_mice.ipynb
combined_periods_1



/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/7.dsl__ipsi_next(WR_vs_LS)_contra_next(WR_vs_LS)__ls_mice.ipynb
combined_periods_1


FileNotFoundError: [Errno 2] No such file or directory: '/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot/left_side_of_the_brain/avg_of_mice/7.dsl__ipsi_next(WR_vs_LS)_contra_next(WR_vs_LS)__ls_mice.ipynb'